In [1]:
import time
import math
import random
import os
from itertools import zip_longest

import numpy as np

import pickle as pkl
import gzip

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.nn.functional as F

from torch import optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils import asMinutes, timeSince, load_zipped_pickle, corpus_bleu, directories
from langUtils import loadLangPairs, langDataset, langCollateFn, initHybridEmbeddings, tensorToList

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import seaborn as sns; sns.set()
sns.set_style("darkgrid")
sns.set_context("paper")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [2]:
data_dir, em_dir = directories()

SPECIAL_SYMBOLS_ID = PAD_ID, UNK_ID, SOS_ID, EOS_ID = 0, 1, 2, 3
NUM_SPECIAL = len(SPECIAL_SYMBOLS_ID)

vi, en = loadLangPairs("vi")

In [3]:
BATCH_SIZE = 64

train_dataset = langDataset([(vi.train_num[i], en.train_num[i]) for i in range(len(vi.train_num)) if (len(vi.train[i]) < vi.max_length) & (len(en.train[i]) < en.max_length)])
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=langCollateFn,
                                           shuffle=True)
dev_dataset = langDataset([(vi.dev_num[i], en.dev_num[i]) for i in range(len(vi.dev_num)) if (len(vi.dev[i]) < vi.max_length) & (len(en.dev[i]) < en.max_length)])
dev_loader = torch.utils.data.DataLoader(dataset=dev_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=langCollateFn,
                                           shuffle=True)

In [4]:
class EncoderRNN(nn.Module):
    """
        Encoder RNN
        Input
            params - Dictionary of paramter
            raw_emb - (100,000, 300) raw embeddings
            learn_ids - list of ids to do embedding learning
            
            inp - (Max Length, Batch Size), original inputs
            inp_lens - (Batch Size), true length of inputs
        Output
            output - (Max Length, Batch Size, Hidden Size), GRU output
            hidden - (2, Batch Size, Hidden Size), Final hidden state of GRU
    """
    def __init__(self, params, raw_emb, learn_ids):
    
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = params['hidden_size']
        self.n_layers = params['n_layers']
        
        self.embedding = initHybridEmbeddings(raw_emb, learn_ids)
        self.gru = nn.GRU(self.embedding.embedding_dim, params['hidden_size'], self.n_layers, bidirectional=True)
        
    def forward(self, inp, inp_lens):
        #Embed input
        embedded = self.embedding(inp)
        #Pack padded
        packed = pack_padded_sequence(embedded, inp_lens).to(device)
        
        #GRU
        output, self.hidden = self.gru(packed)
        #Pad packed
        output, _ = pad_packed_sequence(output)
        #Concat bidirectional layers
        output = output[:, :, :self.hidden_size] + output[:, : ,self.hidden_size:]
        return output, self.hidden
    
class DecoderRNN(nn.Module):
    """
        Decoder RNN
        Input
            params - Dictionary of paramter
            raw_emb - (100,000, 300) raw embeddings
            learn_ids - list of ids to do embedding learning
            
            inp - (1, Batch Size), SOS Token each in batch size
            hidden - (2, 1, Hidden Size), Prev hidden size
        Output
            output - (32, Vocab Size), Probabilities
            hidden - (2, 1 Size, Hidden Size), Final hidden state of GRU
    """
    def __init__(self, params, raw_emb, learn_ids):
        super(DecoderRNN, self).__init__()
        self.hidden_size = params['hidden_size']
        self.n_layers = params['n_layers']
        self.output_size = params['output_size']

        self.embedding = initHybridEmbeddings(raw_emb, learn_ids)
        self.gru = nn.GRU(self.embedding.embedding_dim, params['hidden_size'], self.n_layers, bidirectional=True)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, inp, hidden, encoder_output=None):
        #Embed
        embedded = self.embedding(inp)
        #Dropout
        output = F.relu(embedded)
        
        #Gru
        output, self.hidden = self.gru(output, hidden)
        #Concat directions
        output = output[:, :, :self.hidden_size] + output[:, : ,self.hidden_size:]
        #GRU output to probabilities
        output = torch.exp(self.softmax(self.out(output))).squeeze(0)
        return output, hidden, None

def maskNLLLoss(inp, target, mask):
    """Masked Loss"""
    #Total # real words
    nTotal = mask.sum()
    #Cross entropy
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1).to(device)))
    #Select loss of real words
    loss = crossEntropy.masked_select(mask.to(device)).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [5]:
class Model():
    def __init__(self, encoder, decoder, encoder_optim, decoder_optim):
        
        self.encoder = encoder
        self.decoder = decoder
        
        self.encoder_optim = encoder_optim
        self.decoder_optim = decoder_optim
        
    def fit(self, train_data, dev_data, teacher_forcing_ratio, n_epoch, print_every, n_grams):
        start = time.time()
        
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.n_epoch = n_epoch
        
        
        print("Initializing...")
        start_epoch = 1
        print_loss_total, plot_loss_total = 0 , 0
        plot_losses, plot_train_scores, plot_dev_scores = [], [], []
        
        for epoch in range(start_epoch, n_epoch):
            for i, (inp, inp_lens, output, out_mask, out_max) in enumerate(train_loader):
                loss = self.trainEpoch(inp, inp_lens, output, out_mask, out_max)
                
                print_loss_total += loss
                plot_loss_total += loss
                    
                if i % print_every == 0:
                    train_score = self.bleuScore(train_loader, n_grams)
                    dev_score = self.bleuScore(dev_loader, n_grams)
                    plot_train_scores.append(train_score)
                    plot_dev_scores.append(dev_score)
                    plot_loss_avg = plot_loss_total / print_every
                    plot_losses.append(plot_loss_avg)
                    plot_loss_total = 0       

                    print_loss_avg = print_loss_total / print_every
                    print_loss_total = 0
                    print("Epoch:{} | Time Elapsed:{} | Percent Complete:{:.1} | Loss:{:.4} | TrainScore:{:.4} | DevScore:{:.4}".format(epoch,
                                                                                                                                  timeSince(start, epoch/n_epoch), 
                                                                                                                                  epoch/n_epoch*100, 
                                                                                                                                  print_loss_avg, 
                                                                                                                                  train_score,
                                                                                                                                  dev_score))                

        self.plot_losses = plot_losses
        self.plot_train_scores = plot_train_scores
        self.plot_dev_scores = plot_dev_scores
        return "Training Complete!"            
            
    def trainEpoch(self, inp, inp_lens, output, out_mask, out_max):
        #Zero gradients
        self.encoder_optim.zero_grad()
        self.decoder_optim.zero_grad()

        #Loss vars
        loss, print_losses, n_totals = 0, [], 0
        
        #Encoder Forward
        encoder_output, encoder_hidden = self.encoder(inp, inp_lens)
        
        #Init decoder_input
        decoder_input = torch.LongTensor([[SOS_ID for _ in range(inp.size(1))]]).to(device)
        decoder_hidden = encoder_hidden[:self.decoder.n_layers* (2 *self.decoder.gru.bidirectional)]

        #Teacher Forcing
        if random.random() < self.teacher_forcing_ratio:
            for t in range(out_max):
                #Decoder Forward
                decoder_output, decoder_hidden, _ = self.decoder(decoder_input, decoder_hidden, encoder_output)
                #True Output
                decoder_input = output[t].view(1, -1)
                
                mask_loss, nTotal = maskNLLLoss(decoder_output, output[t], out_mask[t])
                loss += mask_loss
                print_losses.append(mask_loss.item() * nTotal)
                n_totals += nTotal
        else:
            for t in range(out_max):
                decoder_output, decoder_hidden, _ = self.decoder(decoder_input, decoder_hidden, encoder_output)
                _, topi = decoder_output.topk(1)
                decoder_input = torch.LongTensor([[topi[i][0] for i in range(inp.size(1))]]).to(device)
                
                mask_loss, nTotal = maskNLLLoss(decoder_output, output[t], out_mask[t])
                loss += mask_loss
                print_losses.append(mask_loss.item() * nTotal)
                n_totals += nTotal
                
        loss.backward()
        
        _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), 50)
        _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), 50)
        
        self.encoder_optim.step()
        self.decoder_optim.step()

        return sum(print_losses) / n_totals
    
    def bleuScore(self, data_loader, n_grams):
        with torch.no_grad():
            
            true_outputs = []
            decoder_outputs = []
            
            for i, (inp, inp_lens, output, out_mask, out_max) in enumerate(data_loader):
                if i * BATCH_SIZE > 1000:
                    break
            

                for s, seq in enumerate(inp.t()):
                    true_outputs += [[str(w) for w in output[:,s].tolist()]]
        
                    encoder_output, encoder_hidden = self.encoder(seq.view(-1, 1), inp_lens[s].view(1))
        
                    decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_ID
                    decoder_hidden = encoder_hidden[:self.decoder.n_layers* (2 *self.decoder.gru.bidirectional)]

                    decoder_seq_output = [decoder_input.item()]

                    for t in range(out_max):
                        decoder_output, decoder_hidden, _ = self.decoder(decoder_input, decoder_hidden, encoder_output)
                        _, decoder_input = torch.max(decoder_output, dim=1)
                        
                        if (decoder_input.item() == EOS_ID) | (decoder_input.item() == PAD_ID):
                            decoder_seq_output.append(EOS_ID)
                            break
                        
                        decoder_seq_output.append(decoder_input.item())
                        decoder_input = torch.unsqueeze(decoder_input, 0)
                    
                    decoder_seq_output[-1] = EOS_ID
                    decoder_outputs += [[str(w) for w in decoder_seq_output]]
                    
        return corpus_bleu(decoder_outputs, true_outputs, n_grams)

    def showLoss(self):
        plt.figure()
        fig = plt.figure(figsize=(10,6))
        fig_plt = sns.lineplot(x=np.arange(0, self.n_epoch, int(self.n_epoch/len(self.plot_losses))), y=self.plot_losses)
        fig_plt.set_title("Loss Over Time")
        fig_plt.set_ylabel("Loss")
        fig_plt.set_xlabel("Epochs")
        return fig_plt.get_figure()
    
    def showScore(self):
        df = pd.concat([pd.DataFrame({'X':np.arange(0, self.n_epoch, int(self.n_epoch/len(self.plot_losses))), 'Y':self.plot_train_scores, 'Score':'Train'}), 
                        pd.DataFrame({'X':np.arange(0, self.n_epoch, int(self.n_epoch/len(self.plot_losses))), 'Y':self.plot_dev_scores, 'Score':'Dev'})], axis=0)
    
        plt.figure()
        pp = sns.lineplot(data=df, x = 'X', y = 'Y', hue='Score', style="Score", legend= "brief")
        fig_plt.set_title("Score Over Time")
        fig_plt.set_ylabel("Score")
        fig_plt.set_xlabel("Epoch")
        return fig_plt.get_figure()

In [6]:
LEARNING_RATE = 0.001

encoder_params = {'hidden_size':512, 'n_layers':1}
decoder_params = {'hidden_size':encoder_params['hidden_size'], 'n_layers':1, 'output_size':en.n_words}

encoder = EncoderRNN(encoder_params, vi.emb, vi.learn_ids).to(device)
encoder_optim = optim.Adam(encoder.parameters(), lr=LEARNING_RATE)

decoder = DecoderRNN(decoder_params, en.emb, en.learn_ids).to(device)
decoder_optim = optim.Adam(decoder.parameters(), lr=0.001)

In [7]:
model = Model(encoder, decoder, encoder_optim, decoder_optim)
model.fit(train_loader, dev_loader, teacher_forcing_ratio=1.0, n_epoch=10, print_every=400, n_grams=4)

Initializing...
Epoch:1 | Time Elapsed:4m 9s (- 37m 25s) | Percent Complete:1e+01 | Loss:0.0104 | TrainScore:46.82 | DevScore:43.93
Epoch:1 | Time Elapsed:11m 45s (- 105m 47s) | Percent Complete:1e+01 | Loss:3.322 | TrainScore:46.21 | DevScore:43.73
Epoch:1 | Time Elapsed:19m 17s (- 173m 40s) | Percent Complete:1e+01 | Loss:2.938 | TrainScore:45.45 | DevScore:43.66
Epoch:1 | Time Elapsed:26m 50s (- 241m 31s) | Percent Complete:1e+01 | Loss:2.794 | TrainScore:46.99 | DevScore:44.17
Epoch:1 | Time Elapsed:34m 22s (- 309m 21s) | Percent Complete:1e+01 | Loss:2.711 | TrainScore:45.7 | DevScore:43.6
Epoch:2 | Time Elapsed:39m 23s (- 157m 33s) | Percent Complete:2e+01 | Loss:0.7494 | TrainScore:43.99 | DevScore:43.98
Epoch:2 | Time Elapsed:46m 55s (- 187m 42s) | Percent Complete:2e+01 | Loss:2.542 | TrainScore:45.14 | DevScore:44.04
Epoch:2 | Time Elapsed:54m 29s (- 217m 58s) | Percent Complete:2e+01 | Loss:2.513 | TrainScore:46.11 | DevScore:44.32
Epoch:2 | Time Elapsed:62m 1s (- 248m 6s) |

'Training Complete!'

In [50]:
for inp, inp_lens, out, out_mask, out_max in train_loader:
    break

In [20]:
def bleuScore(data_loader):
    with torch.no_grad():
            
        true_outputs = []
        decoder_outputs = []

        for i, (inp, inp_lens, output, out_mask, out_max) in enumerate(data_loader):
            if i * BATCH_SIZE > 100:
                break

            for s, seq in enumerate(inp.t()):
                true_outputs += [[str(w) for w in output[:,s].tolist()]]

                encoder_output, encoder_hidden = encoder(seq.view(-1, 1), inp_lens[s].view(1))

                decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_ID
                decoder_hidden = encoder_hidden[:decoder.n_layers* (2 *decoder.gru.bidirectional)]

                decoder_seq_output = [decoder_input.item()]

                for t in range(out_max):
                    decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_output)
                    _, decoder_input = torch.max(decoder_output, dim=1)
                    decoder_seq_output.append(decoder_input.item())
                    decoder_input = torch.unsqueeze(decoder_input, 0)
                
                decoder_seq_output[-1] = EOS_ID
                decoder_outputs += [[str(w) for w in decoder_seq_output]]
                    
    return decoder_outputs, true_outputs

In [21]:
decoder_outputs, true_outputs = bleuScore(dev_loader)

In [22]:
len(true_outputs)

128

In [23]:
len(decoder_outputs)

128

In [27]:
decoder_outputs[0]

['2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0']

In [91]:
seq = inp.t()[1]
encoder_output, encoder_hidden = encoder(seq.view(-1, 1), inp_lens[1].view(1))
        
decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_ID
decoder_hidden = encoder_hidden[:decoder.n_layers* (2 *decoder.gru.bidirectional)]
        

In [92]:
decoder(decoder_input, decoder_hidden, encoder_output)

(tensor([[1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0', grad_fn=<SqueezeBackward1>),
 tensor([[[-0.0003,  0.0421,  0.0274,  ..., -0.0932, -0.0216, -0.0177]],
 
         [[-0.0174,  0.0321, -0.0225,  ..., -0.0277,  0.0395,  0.0499]]],
        device='cuda:0', grad_fn=<SliceBackward>),
 None)

2

In [45]:
 with torch.no_grad():
            
            true_outputs = []
            decoder_outputs = []
            
            for i, (inp, inp_lens, output, out_mask, out_max) in enumerate(data_loader):
                if i * BATCH_SIZE > len(dev_loader) * BATCH_SIZE:
                    break
                true_outputs += tensorToList(output)
                
                encoder_output, encoder_hidden = self.encoder(inp, inp_lens)
                
                decoder_input = torch.LongTensor([[SOS_ID for _ in range(inp.size(1))]])
                decoder_hidden = 
                
                decoder_batch_outputs = []
                decoder_batch_outputs += decoder_input.tolist()
                
                for t in range(out_max):
                    
                    
                    
                decoder_outputs += tensorToList(torch.tensor(decoder_batch_outputs))

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  61,   61,   30,  ...,    7,  226,   61],
        [  23,   22, 1485,  ...,  674,   81,   99],
        ...,
        [2039,    0,    0,  ...,    0,    0,    0],
        [   6,    0,    0,  ...,    0,    0,    0],
        [   3,    0,    0,  ...,    0,    0,    0]])

In [ ]:
class Attn(torch.nn.Module):
    def __init__(self, params):
        super(Attn, self).__init__()
        self.method = params['method']
        self.hidden_size = params['hidden_size']
        
        #Define extra functions depending on method
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, self.hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, self.hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(self.hidden_size))
    
    def forward(self, hidden, encoder_outputs):
        if self.method == 'general':
            #h_t.T.dot(W_a.dot(h_s))
            attn_energies = torch.sum(hidden * self.attn(encoder_output), dim=2)
        elif self.method == 'concat':
            #v_a * tanh(w_a[h_t,h_s])
            energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
            attn_energies = torch.sum(self.v * energy, dim=2)
        elif self.method == 'dot':
            #h_t.T.dot(h_s)
            attn_energies = torch.sum(hidden * encoder_output, dim=2)

        return F.softmax(attn_energies.t(), dim=1).unsqueeze(1)

class AttnDecoderRNN(nn.Module):
    def __init__(self, params, raw_emb, learn_ids):
        super(AttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = params['attn_model']
        self.hidden_size = params['hidden_size']
        self.output_size = params['output_size']
        self.n_layers = params['n_layers']
        self.dropout = params['dropout']

        # Define layers
        self.embedding = initHybridEmbeddings(raw_emb, learn_ids)
        self.embed_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, self.n_layers, dropout=(0 if n_layers == 1 else dropout), batch_first=True)
        self.concat = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

        self.attn = Attn(self.attn_model, self.hidden_size)

    def forward(self, inp, hidden, encoder_output):
        #Embedding and dropout
        embedded = self.embedding(inp)
        embedded = self.embed_dropout(embedded)
        
        #GRU
        rnn_out, hidden = self.gru(embedded, hidden)
        
        #Attn weights * enc_output
        attn_weights = self.attn(rnn_output, encoder_output)
        context = attn_weights.bmm(encoder_output.transpose(0, 1))
        
        #Concat context to GRU output
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_inp = torch.cat((rnn_out, context), 1)
        concat_out = torch.tanh(self.concat(concat_inp))
        
        #Prediction
        output = self.out(concat_out)
        output = F.softmax(output, dim=1)
        
        return output, hidden